# Sequential Agents

Let's build a purchase scenario, where first we add item to cart, then calculate price and purchase items.

In [ ]:
from typing import TypedDict, List


class Product(TypedDict):
    id: int
    name: str
    stocks: int
    price: float


products: List[Product] = [
    {
        "id": 1,
        "name": "Samsung S26 5G | Purple | 256GB",
        "stocks": 100,
        "price": 78990.00,
    },
    {
        "id": 2,
        "name": "Apple iPhone 15 Pro | Graphite | 128GB",
        "stocks": 50,
        "price": 134900.00,
    },
    {
        "id": 3,
        "name": "OnePlus 12 | Black | 512GB",
        "stocks": 75,
        "price": 69999.00,
    },
    {
        "id": 4,
        "name": "Google Pixel 8 Pro | Obsidian | 128GB",
        "stocks": 40,
        "price": 99999.00,
    },
    {
        "id": 5,
        "name": "Motorola Edge 50 Pro | Luxe Lavender | 256GB",
        "stocks": 120,
        "price": 31999.00,
    },
]


def get_product_by_id(product_id) -> Product | None:
    return next((product for product in products if product["id"] == product_id), None)

In [ ]:
class CartItem(TypedDict):
    id: int
    price: float
    quantity: int
    total_price: float


class AgentState(TypedDict):
    id: int
    name: str
    carts: List[CartItem]
    tax: float
    total_price: float
    final_price: float
    messages: List[str]

In [ ]:
import random


# Add random items into cart
def add_item_to_cart(state: AgentState) -> AgentState:
    """Simple node to add item into cart, picked one item randomly and add this into cart"""

    id = random.randint(1, 5)
    product = get_product_by_id(id)

    if product is None:
        print(f"Product with id {id} not found in our database!")
        return state

    quantity = random.randint(1, 5)
    total_price = product["price"] * quantity

    state["carts"].append(
        {
            "id": product["id"],
            "price": product["price"],
            "quantity": quantity,
            "total_price": total_price,
        }
    )
    state["messages"].append(
        f"Hey {state['name']}, {quantity} '{product['name']}' added into your cart."
    )
    return state


# calculate cart total price
def calculate_cart_total_price(state: AgentState) -> AgentState:
    """Simple node to calculate total price for all items in cart"""

    total_price: float = sum(item["total_price"] for item in state["carts"])
    tax: float = total_price * 18 / 100
    final_price = tax + total_price

    state["total_price"] = total_price
    state["tax"] = tax
    state["final_price"] = final_price
    state["messages"].append(
        f"Hey {state['name']}, current final price of your cart is RS {final_price} with tax RS {tax}"
    )

    return state

In [ ]:
from langgraph.graph import StateGraph, START, END

stateGraph = StateGraph(AgentState)
ADD_TO_CART: str = "ADD_TO_CART"
CART_TOTAL_PRICE: str = "CART_TOTAL_PRICE"

stateGraph.add_node(ADD_TO_CART, add_item_to_cart)
stateGraph.add_node(CART_TOTAL_PRICE, calculate_cart_total_price)

stateGraph.add_edge(START, ADD_TO_CART)
stateGraph.add_edge(ADD_TO_CART, CART_TOTAL_PRICE)
stateGraph.add_edge(CART_TOTAL_PRICE, END)

graph = stateGraph.compile()
graph

In [ ]:
initial_state: AgentState = {
    "id": 12345,
    "name": "SpiderMan",
    "carts": [],
    "total_price": 0,
    "tax": 0,
    "final_price": 0,
    "messages": [],
}

In [ ]:
result = graph.invoke(initial_state)
result